# Locality-Sensitive Hashing and Functions

Prepared by: Norbert Paul R. Gerena

1. Intro/Motivations to LSH
2. LSH Banding technique
    * review of signature Matrix
    * Banding technique
    * Math
        * show intuition of the choice of b and r. ex. b =1 is 100% similar
    
3. Implementation?
4. LS function
    * definition
        * sensitivity
    * And-or combo
    * how about for other distance metrics?
    * effect of number of hash functions to O and to sensitivity
    
5. Examples

## Introduction

The structure and contents of this notebook is mainly based on the first reference [1].

## References

1. Jure Leskovec , Anand Rajaraman, and Jeffrey D Ullman. Mining of massive datasets.
Cambridge university press, 3rd edition, 2019. URL http://www.mmds.org/, chapter 3.